In [1]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np

/Users/giorgiamodi/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PUZZLE_DIM = 3
Action = namedtuple('Action', ['pos1', 'pos2'])

In [3]:
def available_actions(state: np.ndarray) -> list[Action]:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(Action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(Action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: Action) -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [4]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))
state

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 299749.44it/s]


array([[4, 1, 6],
       [3, 0, 5],
       [2, 7, 8]])

## A* Algorithm


In [5]:
def manhattan_distance(state: np.ndarray, goal: np.ndarray) -> int:
    """
    Compute the Manhattan distance heuristic for a given state.
    Args:
        state: Current state of the puzzle as a 2D numpy array.
        goal: Goal state of the puzzle as a 2D numpy array.
    Returns:
        Total Manhattan distance between tiles in 'state' and their positions in 'goal'.
    """
    distance = 0
    for num in range(1, PUZZLE_DIM**2):  # 0 is ignored
        x1, y1 = np.where(state == num)
        x2, y2 = np.where(goal == num)
        distance += abs(x1 - x2) + abs(y1 - y2)
    return distance

In [6]:
def linear_conflict(state: np.ndarray, goal: np.ndarray) -> int:
    manhattan = manhattan_distance(state, goal)
    linear_conflicts = 0

    for row in range(state.shape[0]):
        row_tiles = state[row, :]
        goal_row = goal[row, :]
        for i, tile in enumerate(row_tiles):
            if tile != 0 and tile in goal_row:
                goal_index = np.where(goal_row == tile)[0][0]
                for j in range(i + 1, len(row_tiles)):
                    other_tile = row_tiles[j]
                    if other_tile != 0 and other_tile in goal_row:
                        other_goal_index = np.where(goal_row == other_tile)[0][0]
                        if goal_index > other_goal_index:
                            linear_conflicts += 1

    for col in range(state.shape[1]):
        col_tiles = state[:, col]
        goal_col = goal[:, col]
        for i, tile in enumerate(col_tiles):
            if tile != 0 and tile in goal_col:
                goal_index = np.where(goal_col == tile)[0][0]
                for j in range(i + 1, len(col_tiles)):
                    other_tile = col_tiles[j]
                    if other_tile != 0 and other_tile in goal_col:
                        other_goal_index = np.where(goal_col == other_tile)[0][0]
                        if goal_index > other_goal_index:
                            linear_conflicts += 1

    return manhattan + 2 * linear_conflicts


Numpy arrays are not directly hashable in Python, which means you cannot easily use them as keys in dictionaries or elements in sets. By calling .tobytes(), we convert the numpy array into a byte string (a sequence of bytes), which is hashable and can be used as keys in dictionaries or elements in sets. This allows us to efficiently store and compare puzzle states in explored and parent_map (or other data structures).

In [7]:
from heapq import heappush, heappop

def a_star_solver(start_state: np.ndarray, goal_state: np.ndarray) -> list[np.ndarray]:
    """
    Solve the n^2-1 puzzle using the A* search algorithm.
    Args:
        start_state: The initial scrambled state as a 2D numpy array.
        goal_state: The goal state as a 2D numpy array.
    Returns:
        A list of states (as numpy arrays) representing the solution path, or None if unsolvable.
    """
    # Priority queue: stores tuples (f-score, state)
    frontier = []
    heappush(frontier, (0, start_state.tobytes()))  # Use tobytes() to store the state
    
    # Explored set to prevent revisiting states
    explored = set()
    
    # Parent map for backtracking the solution
    parent_map = {start_state.tobytes(): None}
    
    # Cost map to store the g-scores
    g_score = {start_state.tobytes(): 0}
    
    while frontier:
        # Get the state with the lowest f-score
        _, current_state_bytes = heappop(frontier)
        current_state = np.frombuffer(current_state_bytes, dtype=int).reshape(PUZZLE_DIM, PUZZLE_DIM)
        
        # Check if the current state is the goal
        if np.array_equal(current_state, goal_state):
            # Reconstruct the solution path
            path = []
            while current_state is not None:
                path.append(current_state)
                current_state = parent_map[current_state.tobytes()]
            return path[::-1]  # Reverse the path to start from the initial state
        
        # Mark the current state as explored
        explored.add(current_state_bytes)
        
        # Explore neighbors (valid moves)
        for action in available_actions(current_state):
            neighbor = do_action(current_state, action)
            neighbor_bytes = neighbor.tobytes()
            tentative_g = g_score[current_state_bytes] + 1  # Cost of moving to this neighbor
            
            # If the neighbor is not explored or we found a cheaper path
            if neighbor_bytes not in explored or tentative_g < g_score.get(neighbor_bytes, float('inf')):
                g_score[neighbor_bytes] = tentative_g
                f_score = tentative_g + manhattan_distance(neighbor, goal_state)
                #f_score = tentative_g + linear_conflict(neighbor, goal_state)
                heappush(frontier, (f_score, neighbor_bytes))  # Use the byte representation
                parent_map[neighbor_bytes] = current_state
                
    return None  # Return None if no solution exists


In [8]:
goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

# Solve the puzzle using A*
solution_path = a_star_solver(state, goal_state)

# Display the solution steps
if solution_path:
    for step, s in enumerate(solution_path):
        print(f"Step {step}:\n{s}")
else:
    print("No solution found!")


Step 0:
[[4 1 6]
 [3 0 5]
 [2 7 8]]
Step 1:
[[4 1 6]
 [0 3 5]
 [2 7 8]]
Step 2:
[[0 1 6]
 [4 3 5]
 [2 7 8]]
Step 3:
[[1 0 6]
 [4 3 5]
 [2 7 8]]
Step 4:
[[1 3 6]
 [4 0 5]
 [2 7 8]]
Step 5:
[[1 3 6]
 [4 5 0]
 [2 7 8]]
Step 6:
[[1 3 0]
 [4 5 6]
 [2 7 8]]
Step 7:
[[1 0 3]
 [4 5 6]
 [2 7 8]]
Step 8:
[[0 1 3]
 [4 5 6]
 [2 7 8]]
Step 9:
[[4 1 3]
 [0 5 6]
 [2 7 8]]
Step 10:
[[4 1 3]
 [2 5 6]
 [0 7 8]]
Step 11:
[[4 1 3]
 [2 5 6]
 [7 0 8]]
Step 12:
[[4 1 3]
 [2 0 6]
 [7 5 8]]
Step 13:
[[4 1 3]
 [0 2 6]
 [7 5 8]]
Step 14:
[[0 1 3]
 [4 2 6]
 [7 5 8]]
Step 15:
[[1 0 3]
 [4 2 6]
 [7 5 8]]
Step 16:
[[1 2 3]
 [4 0 6]
 [7 5 8]]
Step 17:
[[1 2 3]
 [4 5 6]
 [7 0 8]]
Step 18:
[[1 2 3]
 [4 5 6]
 [7 8 0]]


## DFS with Bound

In [9]:
def depth_first_search_with_bound(start_state: np.ndarray, goal_state: np.ndarray, depth_limit: int) -> list[np.ndarray]:
    """
    Perform a Depth-First Search with a bound to solve the n^2-1 puzzle.
    Args:
        start_state: The initial scrambled state as a 2D numpy array.
        goal_state: The goal state as a 2D numpy array.
        depth_limit: The maximum depth to explore.
    Returns:
        A list of states representing the solution path, or None if unsolvable within the depth limit.
    """
    # Stack for DFS: stores tuples of (state, current path, current depth)
    stack = [(start_state, [], 0)]
    
    # Set for explored states (to avoid revisiting)
    explored = set()
    
    while stack:
        current_state, path, depth = stack.pop()
        
        # If we have reached the goal, return the solution path
        if np.array_equal(current_state, goal_state):
            return path + [current_state]
        
        # If we exceed the depth limit, stop exploring this path
        if depth >= depth_limit:
            continue
        
        # Add the current state to the explored set
        current_state_bytes = current_state.tobytes()
        if current_state_bytes in explored:
            continue
        explored.add(current_state_bytes)
        
        # Explore all valid actions (neighbors)
        for action in available_actions(current_state):
            neighbor = do_action(current_state, action)
            
            # Add the neighbor state to the stack with an incremented depth
            stack.append((neighbor, path + [current_state], depth + 1))
    
    return None  # Return None if no solution is found within the depth limit


def iterative_deepening_dfs(start_state: np.ndarray, goal_state: np.ndarray, max_depth: int) -> list[np.ndarray]:
    """
    Perform Iterative Deepening DFS to solve the n^2-1 puzzle.
    Args:
        start_state: The initial scrambled state as a 2D numpy array.
        goal_state: The goal state as a 2D numpy array.
        max_depth: The maximum depth to explore.
    Returns:
        A list of states representing the solution path, or None if unsolvable within the maximum depth.
    """
    for depth_limit in range(max_depth + 1):
        print(f"Trying depth limit: {depth_limit}")
        result = depth_first_search_with_bound(start_state, goal_state, depth_limit)
        if result:
            return result
    return None  # Return None if no solution is found within the max depth

In [10]:
# Solve with a depth bound (max_depth = 25)
solution_path = iterative_deepening_dfs(state, goal_state, max_depth=25)

if solution_path:
    print("Solution found!")
    for step, s in enumerate(solution_path):
        print(f"Step {step}:\n{s}")
        
else:
    print("No solution found within the depth limit.")

Trying depth limit: 0
Trying depth limit: 1
Trying depth limit: 2
Trying depth limit: 3
Trying depth limit: 4
Trying depth limit: 5
Trying depth limit: 6
Trying depth limit: 7
Trying depth limit: 8
Trying depth limit: 9
Trying depth limit: 10
Trying depth limit: 11
Trying depth limit: 12
Trying depth limit: 13
Trying depth limit: 14
Trying depth limit: 15
Trying depth limit: 16
Trying depth limit: 17
Trying depth limit: 18
Trying depth limit: 19
Trying depth limit: 20
Solution found!
Step 0:
[[4 1 6]
 [3 0 5]
 [2 7 8]]
Step 1:
[[4 1 6]
 [3 5 0]
 [2 7 8]]
Step 2:
[[4 1 0]
 [3 5 6]
 [2 7 8]]
Step 3:
[[4 0 1]
 [3 5 6]
 [2 7 8]]
Step 4:
[[4 5 1]
 [3 0 6]
 [2 7 8]]
Step 5:
[[4 5 1]
 [0 3 6]
 [2 7 8]]
Step 6:
[[4 5 1]
 [2 3 6]
 [0 7 8]]
Step 7:
[[4 5 1]
 [2 3 6]
 [7 0 8]]
Step 8:
[[4 5 1]
 [2 3 6]
 [7 8 0]]
Step 9:
[[4 5 1]
 [2 3 0]
 [7 8 6]]
Step 10:
[[4 5 1]
 [2 0 3]
 [7 8 6]]
Step 11:
[[4 0 1]
 [2 5 3]
 [7 8 6]]
Step 12:
[[4 1 0]
 [2 5 3]
 [7 8 6]]
Step 13:
[[4 1 3]
 [2 5 0]
 [7 8 6]]
Ste